# Cat vs Dog

In this project, we will go through the process of creating a cat vs dog classifier using UpStride Classification API.

First, let's download the dataset by visiting the Kaggle competition https://www.kaggle.com/c/dogs-vs-cats/data and clicking the "download all" button.

Unzip the archive. You will find another zip file inside, "train.zip". Unzip this one also.

Now, let's prepare the execution environment. The simplest way is using docker. Run at the root of the project:

docker run -it --rm --gpus all \
    -v $(pwd):/opt \
    -v /path/to/your/train:/dataset \
    -v /path/to/save/your/experiment:/experiment \
    -p 127.0.0.1:8889:8888
    upstride/classification_api:tensorflow-1.0 /bin/bash -c "pip install notebook && jupyter notebook --ip=0.0.0.0 --allow-root"

a new jupyter notebook should appear, please continue the tutorial there :)

Now we can prepare the dataset by moving the images in two differents folder for the two classes


In [1]:
# change this path to match you directory
train_directory = '/dataset'
cat_directory = '/experiment/sorted_dataset/cat'
dog_directory = '/experiment/sorted_dataset/dog'

In [2]:
import os
import shutil

os.makedirs(cat_directory, exist_ok=True)
os.makedirs(dog_directory, exist_ok=True)

for image in os.listdir(train_directory):
    if image.startswith('cat'):
        shutil.copy(os.path.join(train_directory, image), cat_directory)
    else:
        shutil.copy(os.path.join(train_directory, image), dog_directory)

Now we can convert this dataset to tfrecord format. For this we will use the script tfrecord_writer.py in scripts directory

In [3]:
!cd /opt/scripts \
    && export PYTHONPATH=$PYTHONPATH:$(pwd)/.. \
    && python3 tfrecord_writer.py \
      --tfrecord_dir_path /experiment/tfrecord \
      --name cat_dog_dataset \
      --preprocessing CENTER_CROP_THEN_SCALE \
      --image_size 256 256 \
      --data.images_dir_path /experiment/sorted_dataset/ \
      --data.split_names train validation \
      --data.split_percentages 0.7 0.3

2020-10-02 17:16:42.065588: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Building tfrecord for dataset named cat_dog_dataset .....
Found 2 classes
Creating train split.....
Done
Creating validation split.....
Done
Dataset creation complete and stored in /experiment/tfrecord


if you go to the directory /home/seb/Downloads/dogs-vs-cats/tfrecord you will find 4 files :
 - dataset_info.yaml
 - train_0.tfrecord
 - train_1.tfrecord
 - validation_0.tfrecord
 
in dataset_info you will see information regarding this dataset.

In this case, you can see that the training dataset has 17500 images and the validation 7500

Now we can start a real training :

In [ ]:
!cd /opt \
    && python3 train.py \
      --model_name MobileNetV2 \
      --num_epochs 20 \
      --checkpoint_dir /experiment/checkpoint \
      --log_dir /experiment/log \
      --framework tensorflow \
      --export.dir /experiment \
      --dataloader.name cat_dog_dataset \
      --dataloader.data_dir /experiment/tfrecord \
      --input_size 224 224 3 \
      --dataloader.train_list RandomHorizontalFlip Normalize \
      --dataloader.val_list Normalize \
      --dataloader.batch_size 64 \
      --num_classes 2 \
      --optimizer.lr 0.1

2020-10-02 17:24:51.555062: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
{'dataloader': {'train_list': ['RandomHorizontalFlip', 'Normalize'], 'val_list': ['Normalize'], 'data_dir': '/experiment/tfrecord', 'name': 'cat_dog_dataset', 'train_split_id': 'train', 'val_split_id': 'validation', 'batch_size': 64, 'Normalize': {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225], 'scale_in_zero_to_one': True, 'only_subtract_mean': False}, 'ColorJitter': {'brightness': 0.05, 'contrast': [0.7, 1.3], 'saturation': [0.6, 1.6], 'hue': 0.08, 'clip': [0.0, 1.0]}, 'RandomRotate': {'angle': 10, 'interpolation': 'nearest'}, 'CentralCrop': {'size': [224, 224], 'crop_proportion': 0.875, 'interpolation': 'bicubic'}, 'RandomCrop': {'size': [224, 224, 3]}, 'Resize': {'size': [224, 224], 'interpolation': 'bicubic'}, 'ResizeThenRandomCrop': {'size': [256, 256], 'crop_size': [224, 224, 3], 'interpolation': 'bicubic'}, 'RandomCropThe

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 226, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

block_9_depthwise_relu (ReLU)   (None, 14, 14, 384)  0           block_9_depthwise_BN[0][0]       
__________________________________________________________________________________________________
block_9_project (Conv2D)        (None, 14, 14, 64)   24576       block_9_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_9_project_BN (BatchNormal (None, 14, 14, 64)   256         block_9_project[0][0]            
__________________________________________________________________________________________________
block_9_add (Add)               (None, 14, 14, 64)   0           block_8_add[0][0]                
                                                                 block_9_project_BN[0][0]         
__________________________________________________________________________________________________
block_10_expand (Conv2D)        (None, 14, 14, 384)  24576       block_9_add[0][0]                
__________

Epoch 1/20
2020-10-02 17:24:57.776698: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2020-10-02 17:24:58.356690: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
      2/Unknown - 0s 208ms/step - loss: 2.3608 - accuracy: 0.5234 - top_k_categorical_accuracy: 1.0000WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0902s vs `on_train_batch_end` time: 0.3264s). Check your callbacks.
274/274 [==============================] - 124s 452ms/step - loss: 1.5405 - accuracy: 0.6134 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.3461 - val_accuracy: 0.6728 - val_top_k_categorical_accuracy: 1.0000
Epoch 2/20
274/274 [==============================] - 123s 448ms/step - loss: 1.2322 - accuracy: 0.7154 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.2362 - val_accuracy: 0.6939 - val_top_k_categorical_accura